In [1]:
%%html
<style>
table {float:right}
</style>

# Testing In Python
|        |                           |
|-------:|:--------------------------|
|**By:** | Mohammad Abouali, Ph.D.   |
|        | Software Eng./Prog. III.  |
|        | VAST/TDD/CISL/NCAR/UCAR   |
|        | E-Mail: mabouali@ucar.edu |
|        | Office: 303-497-1893      |


# Table of Contents
- [Different Types of Testing](#DifferentTypesOfTesting)
- [Testing Frameworks](#TestingFrameworks)
    - [Manual Testing](#ManualTesting)
    - [doctest](#doctest)
    - [unittest](#unittest)
        - [setUp and tearDown](#setUp_tearDown)
        - [One Last Thing - FunctionTestCase](#unittest_OneLastThing)
        - [Where to go?](#unittest_wheretogo)
    - [pytest](#pytest)
- [A note on code coverage](#ANoteOnCodeCoverage)
- [One Last Thing - Using IDEs](#OneLastThing)
    
    

<a id="DifferentTypesOfTesting"></a>
# Different Types of Testing

You are not done with your code, until you have ran the tests.

There are different type of tests that you could perform. The most relevant ones for our community are:

-  **Unit Test:** In this test, you are checking each sub-component of your code to make sure that it is working as it is expected. While Unit testing, you should isolate the code being tested in such a way that only one part of the code is being tested; hence, the name unit testing. If it is not possible to do that, i.e. isolating the code and testing it independently, then this is a good omen or sign that you need to refactor your code.
-  **Performance Test:** In this test, you are looking to see how the performance of the system is using different type of inputs and size of data. If you are testing a micro-service, you could also check how the system behaves as the number of simultanous users accessing the micro-service increases (Different goal than stress-testing).
-  **Integration Test:** In this test, you are making sure that different components of your code are able to work together without any problem.
-  **Regression Test:** In this test, you are looking how the new updates and changes in the code, is going to change the behavior of the system. Is one component becoming slower? Are new bugs being discovered? and similar questions like these.

<a id="TestingFrameworks"></a>
# Testing Frameworks
Within python environment, there are several testing framework available. Depending on your code, and, more importantly, your preferences, one framework may be more suitable for your work than the others. You are not limited to use only one framework in your project, although preferably do if you could. But, you could use multiple of them. These frameworks could be divided into two categories:

* **Built-In** which are shipped as part of the standard python library; hence, if you install python, you would have access to them.
* **3rd-party** frameworks, which you need to install them separately. However, their installation is as easy as installing any other libraries within python. So, this is hardly a valid reason to not use them.

Usually the 3rd-party testing framework, provide support for the built-in testing frameworks. This means that if you have some tests written using built-in testing frameworks, if for any reason at some point in your project time-line, you decide to use a 3rd-party testing frame, there is a high chance that you could reuse all the built-in tests that you have already coded within the 3rd-party framework without any rewriting. So, it might not be a bad idea to start with one of the built-in frameworks and if it was necessary, switch to a 3rd-party, later.

<a id = "ManualTesting"></a>
## 2.1 Manual Testing (Not Recommended)
Imagine you have the following code to convert Celsius to Fahrenheit (well, not that hard to imagine it, you have the code listed below):

In [2]:
def convert_celsius_to_fahrenheit(celsius: float) -> float:
    if celsius<273.15:
        raise ValueError("Isn't that a bit too cold?")
    fahrenheit = (celsius * 9.0 / 5.0) + 32
    return fahrenheit

Now you could write your own tests to make sure that your function is working properly.

In [3]:
def test_convert_celsius_to_fahrenheit():
    fahrenheit = convert_celsius_to_fahrenheit(0);
    if (fahrenheit == 32.0):
        print(f"0 degree Celsius is {fahrenheit} fahrenheit. Passed.")
    else:
        print(f"0 degree Celsius is not {fahrenheit} fahrenheit. Failed.")

Now we could run our test manually and see the output and how beautifully it converts 0 degree Celsius to 32 degree Fahrenheit. Let's do it:

In [4]:
test_convert_celsius_to_fahrenheit()

ValueError: Isn't that a bit too cold?

mmm?! Our code generated an error. It raised ```ValueError``` exception complaining the temperature is too cold!!! If you check the code you will notice that we have missed a negative sign when checking for absolute zero. Let's fix it.

In [5]:
def convert_celsius_to_fahrenheit(celsius: float) -> float:
    if celsius<-273.15:
        raise ValueError("Isn't that a bit too cold?")
    fahrenheit = (celsius * 9.0 / 5.0) + 32
    return fahrenheit

Now let's run the test again:

In [6]:
test_convert_celsius_to_fahrenheit()

0 degree Celsius is 32.0 fahrenheit. Passed.


Great. The test passed.

This is an example, and, of course, it was easy to spot the error without even needing to write any tests. However, sometimes (and by "sometimes", I mean most of the time), the logic of your code is more complicated than a simple unit conversion. It's possible that making a little change in one part of your code, might end up causing problem in another part of the same function or method, or another part of your project and essentially causing troubles and issues (this has nothing to do with butterfly effect).

Are we done with testing our code? No. We wrote only one test, and if you check carefully, you would notice that our only test does not end up with executing/running the code inside the if-block. 

Let's also write a test to make sure that our if-block is also executed in our tests and we do get at least 100% code coverage in our tests. (suspicious why I said "at least 100% code coverage"? What else is there to test if we have already tested 100% lines of our codes?!! Good catch! We will get to that later!)

In [7]:
def test2_convert_celsius_to_fahrenheit():
    try:
        convert_celsius_to_fahrenheit(-274)
        print("Somehow we think -274 celsius is ok. Failed.")
    except ValueError:
        print("our conversion detected that -274 is too cold. Passed.")

Now we run our test:

In [8]:
test2_convert_celsius_to_fahrenheit()

our conversion detected that -274 is too cold. Passed.


Note that in the second test, we checked that our function/method would generate error. In this case, generating error was the correct output. So, sometimes we check if the code works properly without any error, and sometimes, we do need to check and make sure that the code is actually going to generate error. Generating error is the correct output in these cases.

By the way, you do properly check the input and convey the message if anything is wrong, right? You do error handling, correct?

<a id = "doctest"></a>
## doctest (better than no test at all)
In previous section, we manually wrote some tests and manually executed them. As our code increases, we would have many of these test functions and we won't be able to run them one by one and check if they passed. So, we might be tempted to write a script that manually executes all our tests, parses the output, and makes sure that everything is ok. If not, that script would summarize which tests failed, so that we could fix our code.

Before pouring yourself coffee (or whatever that gets you going) and start coding such system, Let me remind you of a harsh truth: "Your situation is not that unique as you think it is". This applies in this case too. That's why many of these testing frameworks were created. I mean you are actually sitting in a session that talks about testing (or reading this document about testing! same thing); that should be a pretty good hint that someone else had this issue before. So, before starting from the scratch, let's look at what has been already done, so if our situtation turned out to be indeed unique, maybe we could add a new feature to one of these testing framework to support the unique thing that you want to do. By the way, in research don't you first perform a literature review?

```doctest``` is one of those frameworks. The good thing about the doctest is that, you write your documentation for your code and the tests at the same time. So, this is "buy two for the price of one" type of the situation (don't shoot birds, please!).

Let's see how ```doctest``` works:

In [9]:
def convert_celsius_to_fahrenheit(celsius: float) -> float:
    """
    converts temperature from Celsius to Fahrenheit. 
    
    Examples:
    >>> convert_celsius_to_fahrenheit(0)
    32.0
    
    If you enter any number lower than -273.15 Celsius, the code
    will generate a ValueError:
    >>> convert_celsius_to_fahrenheit(-274)
    Traceback (most recent call last):
    ...
    ValueError: Isn't that a bit too cold?
    """
    if celsius<-273.15:
        raise ValueError("Isn't that a bit too cold?")
    fahrenheit = (celsius * 9.0 / 5.0) + 32
    return fahrenheit

As you can see, I provided some documentation for the method and provided some examples. The magic of ```doctest``` is that it will detect the examples you provide in your documentation and turn them into tests. Now let's execute doctest and have it test our code:

In [10]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

If you want to get more information you could turn the verbose mode on:

In [11]:
doctest.testmod(verbose=True)

Trying:
    convert_celsius_to_fahrenheit(0)
Expecting:
    32.0
ok
Trying:
    convert_celsius_to_fahrenheit(-274)
Expecting:
    Traceback (most recent call last):
    ...
    ValueError: Isn't that a bit too cold?
ok
3 items had no tests:
    __main__
    __main__.test2_convert_celsius_to_fahrenheit
    __main__.test_convert_celsius_to_fahrenheit
1 items passed all tests:
   2 tests in __main__.convert_celsius_to_fahrenheit
2 tests in 4 items.
2 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=2)

Since we are in Jupyter Notebook, we manually executed the ```doctest.testmod()```. However, outside jupyter notebook, there are other options available to execute doctest. One of them is to add a special if-block at the end of your code:

In [12]:
# Let's say this is the start of your python file that you store your methods.
# Let's call this file, conversion_utilities.py

def convert_celsius_to_fahrenheit(celsius: float) -> float:
    """
    converts temperature from Celsius to Fahrenheit. 
    
    Examples:
    >>> convert_celsius_to_fahrenheit(0)
    32.0
    
    If you enter any number lower than -273.15 Celsius, the code
    will generate a ValueError:
    >>> convert_celsius_to_fahrenheit(-274)
    Traceback (most recent call last):
    ...
    ValueError: Isn't that a bit too cold?
    """
    if celsius<-273.15:
        raise ValueError("Isn't that a bit too cold?")
    fahrenheit = (celsius * 9.0 / 5.0) + 32
    return fahrenheit

if __name__ == "__main__":
    import doctest
    doctest.testmod()
# End of conversion_utilities.py file

Now to execute the tests, on your command prompt you could simply type:

```
python conversion_utilities.py
```

If you want to enable the verbose mode you could issue:

```
python conversion_utilities.py -v
```

The alternate method is:

```
python -m doctest -v conversion_utilities.py
```

Note that this time, we had ```-v``` before the python file.

<a id = "unittest"></a>
## unittest
Python standard library includes a module called ```unittest```, which is inspired by ```JUnit```, a test framework for Java programming language, which is dominantly used through out the industry (well, dominantly, if their code is JVM and Java based, of course).

Let's see how we write a unittest and how we execute them.

In [13]:
# First we need to import the unittest module. This module is shipped with
# python's standard library. So, you should not need to install it separately.
import unittest

# Create a test class for module or package that you want to test.
# The names are arbitrary. It doesn't need to have the name of the function
# in its name. You could use the name and test cases to separate and group your
# tests.
#
# As you can see, the requirement is that your class should be extending TestCase
class Test_Conversion_Utilities(unittest.TestCase):
    # Now write your tests as a method in this class. The method name
    # should start with "test", preferably. This causes the unit test
    # to distinguish test methods from the none-test methods.
    # It is possible to have test methods that their name does not start with
    # "test". Otherwise, you need to provide more instruction for unittest
    # to find them.
    def test_converting_0_celsius(self):
        fahrenheit = convert_celsius_to_fahrenheit(0)
        expected = 32.0;
        self.assertEqual(expected, fahrenheit)
    
    def test_converting_n40_celsius(self):
        fahrenheit = convert_celsius_to_fahrenheit(-40)
        expected = -40.0;
        self.assertEqual(expected, fahrenheit)
        
    def test_tooCold(self):
        with self.assertRaises(ValueError):
            convert_celsius_to_fahrenheit(-274)

Now let's run the tests. First we manually execute the tests in jupyter notebook, and then explain how to do it on the command line:

In [14]:
unittest.main(argv=[""],exit=False);

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


if you want to run it in verbose mode do:

In [15]:
unittest.main(argv=[""],exit=False, verbosity=2);

test_converting_0_celsius (__main__.Test_Conversion_Utilities) ... ok
test_converting_n40_celsius (__main__.Test_Conversion_Utilities) ... ok
test_tooCold (__main__.Test_Conversion_Utilities) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


The alternative is (again) to add the following if-block to the end of your file:

```python
if __name__ == '__main__':
    unittest.main()
```

Then as before if you issue:

```
python mytestcase.py
```

the unit test is executed.

However, you do not need to have that if-block at the end of your python file. You could also issue:

```
python -m unittest mytestcase.py
```

which causes the unittest to run.

However, the more common way of running tests is to have unittest discover them automatically and run them all for you:

```
python -m unittest discover
```

This will cause unittest to search for all the tests, execute them, and summarize the results for you.

<a id = "setUp_tearDown"></a>
### Test Set Up and Tear Down

Some times, before each test, there are certain setting up are needed. The code for the setup before each test is the same. So, one option is to put the setup code in each of the tests function. But that goes against DRY code principal (DRY as in **D**on't **R**epeat **Y**ourself). I know water is life but let's keep it out of our codes and avoid WET codes (again, WET as in **W**rite **E**verything **T**wice, or even more)

The same goes for after the test is done. There might be certain tasks that you want to do after each test. And, as mentioned above, we don't want to copy paste those codes in all our tests.

Fortunately, ```unittest``` allows you to do that by providing special functions within your test case. They are called: ```setUp()``` and ```tearDown()```.  Let's see how it works:

In [16]:
import unittest
class Test_Conversion_Utilities(unittest.TestCase):
    def setUp(self):
        print("I will be executed before each test.")
    
    def tearDown(self):
        print("I will be executed after each test.");
    
    def test_converting_0_celsius(self):
        print("Testing conversion of 0 Celsius ...")
        fahrenheit = convert_celsius_to_fahrenheit(0)
        expected = 32.0;
        self.assertEqual(expected, fahrenheit)
    
    def test_converting_n40_celsius(self):
        print("testing conversion of -40 celsius ...")
        fahrenheit = convert_celsius_to_fahrenheit(-40)
        expected = -40.0;
        self.assertEqual(expected, fahrenheit)
        
    def test_tooCold(self):
        print("Testing impossibly cold condition ...")
        with self.assertRaises(ValueError):
            convert_celsius_to_fahrenheit(-274)

Now let's run our test:

In [17]:
unittest.main(Test_Conversion_Utilities(), argv=[""],exit=False, verbosity=2);

test_converting_0_celsius (__main__.Test_Conversion_Utilities) ... ok
test_converting_n40_celsius (__main__.Test_Conversion_Utilities) ... ok
test_tooCold (__main__.Test_Conversion_Utilities) ... 

I will be executed before each test.
Testing conversion of 0 Celsius ...
I will be executed after each test.
I will be executed before each test.
testing conversion of -40 celsius ...
I will be executed after each test.
I will be executed before each test.
Testing impossibly cold condition ...
I will be executed after each test.


ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


Great; But what if I want to run some codes before the tests, but I don't need to do it before each tests. For example, let's say, our tests include writing some files to the disk. I want to creat a folder and write some data which is produced during the test into a file. So, I need to first prepare that folder. But I don't need to do that before each test!I can do that once, and all the tests in that test case could use the same folder.

The same goes for after the tests. Let's say, in our example, once all the tests are done. I could go and clean the out files generated during the tests and remove the folder I created for the test. Well, we could do this after all tests are done.

Here how you could do this in ```unittest```, using ```setUpClass()``` and ```tearDownClass()```:

In [18]:
class Test_Conversion_Utilities(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        print("I will be executed only once before all tests.")
    
    @classmethod
    def tearDownClass(cls):
        print("I will be executed only once after all tests.")
        
    def setUp(self):
        print("I will be executed before each test.")
    
    def tearDown(self):
        print("I will be executed after each test.");
    
    def test_converting_0_celsius(self):
        print("Testing conversion of 0 Celsius ...")
        fahrenheit = convert_celsius_to_fahrenheit(0)
        expected = 32.0;
        self.assertEqual(expected, fahrenheit)
    
    def test_converting_n40_celsius(self):
        print("testing conversion of -40 celsius ...")
        fahrenheit = convert_celsius_to_fahrenheit(-40)
        expected = -40.0;
        self.assertEqual(expected, fahrenheit)
        
    def test_tooCold(self):
        print("Testing impossibly cold condition ...")
        with self.assertRaises(ValueError):
            convert_celsius_to_fahrenheit(-274)

In [19]:
unittest.main(Test_Conversion_Utilities(), argv=[""],exit=False, verbosity=2);

test_converting_0_celsius (__main__.Test_Conversion_Utilities) ... ok
test_converting_n40_celsius (__main__.Test_Conversion_Utilities) ... ok
test_tooCold (__main__.Test_Conversion_Utilities) ... 

I will be executed only once before all tests.
I will be executed before each test.
Testing conversion of 0 Celsius ...
I will be executed after each test.
I will be executed before each test.
testing conversion of -40 celsius ...
I will be executed after each test.
I will be executed before each test.
Testing impossibly cold condition ...
I will be executed after each test.
I will be executed only once after all tests.


ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


pay extra attention to how ```setUpClass()``` and ```tearDownClass()``` are defined. First, they receive a ```cls``` input, not ```self```. What you do there is valid for the entire class. not a specific instantiation of the class. Second, you need to decorate them with ```@classmethod```.

<a id = "unittest_OneLastThing"></a>
### One last thing (as Steve Jobs used to say)
Before wrapping our discussion on ```unittest```, I want to cover one more thing, and that's ```FunctionTestCase```. 

Let's say you have already written some functions to test your code. And you want to use them now in ```unittest```. You could easiy create a new test case out of these functions, by using ```FuntionTestCase```. Here is the command:

```python
testcase = unittest.FunctionTestCase(yourtestfunctionname, setUp=yoursetupFunction, tearDown=yourteardownfunction)
```

Although, the python's standard library warns against doing this, which I agree. I think having some tests using common precodure, is still better than manual tests, and, of course, far better than no tests at all.

<a id = "unittest_wheretogo"></a>
### Where to go from here
We just examined one component of ```unittest``` and that was the ```TestCase```. However, there is also test fixtures, suits, and runner. Even for ```TestCase``` we just covered the most basic features, there are a lot to it than what was just covered here. Although we covered the basics of ```unittest``` and ```TestCase``` here, but using even these basic features, you could already create a lot of tests and in many cases properly test your codes. So donot downplay their power, because they are basics. There is already a lot you could do with them.

If you are interested in learning more about ```unittest``` and all its feature, it is strongly suggested to follow
```unittest``` documentation available [here](https://docs.python.org/3/library/unittest.html#module-unittest).

<a id = "pytest"></a>
## pytest
Another commonly used framework for testing is ```pytest```. This is a 3rd-party framework and does not ship with python's standard library. However, it is easy to install it using pip:

```
pip install -U pytest
```

or anaconda:

```
conda install -c anaconda pytest
```

```pytest``` is compatible with ```unittest```, meaning that if you have only ```unittest``` tests written for you code, by going into your project directory and issueing ```pytest``` at the command prompt, it will automatically detects all your tests written in ```unittest``` and executes them.

if you want to write some ```pytest```, at the most simplest form is to just write some additional function which its name starts with **test**. Then issueing the following command would cause ```pytest``` to run your tests:

```
pytest your_filename_including_functions_starting_with_test.py
```

Or even better, if you name your test files to either start with ```test_``` or end with ```_test.py```, pytest automatically detects them and executes all the functions in them which their name starts with ```test```.

```pytest``` comes with some ready to use fixtures. For example ([ref](https://docs.pytest.org/en/latest/getting-started.html)):

```python
def test_needsfiles(tmpdir):
    print(tmpdir)
    #Now do things
    # assert
```

Once executed, ```pytest``` will automatically creates temporary directory for you, which you could use in your tests. ```pytest``` comes with many more ready to use fixtures.

<a id = "ANoteOnCodeCoverage"></a>
# A Note on Code Coverage

As promissed earlier, we are getting back to code coverage. Essentially, the code coverage in testing refers to what part of the code get executed during the tests. If all part of the codes get executed at least once, you have 100% code coverage.

Having 100% code coverage is great. But having 100% code coverage doesn't mean that you have covered everything. How so? Consider the following code flow:

![A code flow with conditional parts](CodeCoverage.png "Conceptual Code")

As you can see in the figure, the code first executes ```P1```; then it checks a condition, and depending on the condition, it executes one of the ```P2.a```, ```P2.b```, or ```P2.c```. Then it executes ```P3```, after which, checks another condition. Again, depending on the outcome of the condition it would execute either ```P4.a```, ```P4.b```, ```P4.c```, or ```P4.d```. Finaly, it would execute ```P5``` and the code finishes. To cover all part of the code, i.e. 100% code coverage, theoretically, you would need only 4 tests. Each test covers one path in the code and after all the 4 tests, all code nodes of this graph is visited; hence, you will get 100% code coverage in your testing. But as you can see in the code, theoretically, there are 12 different paths in this code. Hence, theoretically, to fully test all the paths that are possible you need 12 tests.

That's why having 100% code coverage is very good, but that's not the end of the story.

If you are interested, there is a 3rd-party tool called [Coverage.py](https://coverage.readthedocs.io/en/v4.5.x/) which you could use for tracking code coverage in your testing.

<a id = "OneLastThing"></a>
# One Last Thing (No, really, this is the last one) -> Using IDEs
There is no shame in using Integrated Development Environments (IDEs). If you are comfortable with your vi or emacs environment, fine. But IDE's do provide tremendous help. I am not going into the detail of how you could use IDEs and how they can be helpful. But one of the IDEs that I like for Python programming is [PyCharm](https://www.jetbrains.com/pycharm) provided by JetBrains.

PyCharm has a community version which is free and it has a professional version, which you have to pay. If you are student and studying in an accredited university, then you are in luck. You could get most of their products, including the professional versions free of charge.

PyCharm provides a lot of hint, about your code. It warns you if you are following Python style guide. If you are coding following object oriented programming principal, it provides a lot of utilities that could help. It has a very powerfull visual debugger. And it supports the main testing frameworks. Here is an example:

![A Python unittest test case in PyCharm](PyCharmIDECode.png "PyCharm")

If you notice, there are some green arrows next to the ```class``` definition on line 6, and some more green arrows at line 21 (we won), 27, and 33, where we have defined our test functions. By simply clicking on these green arrows, you are able to run all the tests in that test case or just run one specific test in that test case. Once the tests ran, you could easilly switch to each test and see the result or check which one failed and investigate it further. The following is the example screen shot of running tests:

![PyCharm summarizing tests results](PyCharmTestResults.png "PyCharm Test Results")
